In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jul  8 14:53:57 2022

@author: hannahharris
"""

import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
import copy
from gtfparse import read_gtf

#dictionary with gene and then a SET of the start sites 
#negative or positive
import os
os.getcwd()
#os.chdir('/Users/hannahharris/Documents/996355730663085184-web-restore_2/Documents/') 

def get_all_comps(transcript_coords): 
    '''
    #https://stackoverflow.com/questions/65181109/recursively-find-all-combinations-of-list 
    takes in a list of anything (transcripts, starts)
    returns a list of all the comparisons  
    '''
    if not transcript_coords:
       return [[]]
    else:
        all_comps = get_all_comps(transcript_coords[:-1])
        all_comps_1 = all_comps + [c + [transcript_coords[-1]] for c in all_comps]
        return all_comps_1 
    
def get_right_length_comps(all_comps):
    '''
    '''    
    return [x for x in all_comps if len(x) == 2]


def get_all_starts(table):
    '''
    take a table (gtf like  format)
    returns dictionnary of genes, starts, and transcripts
    '''
    gene_dict = {}
    for row in range(len(table)):
        #print(table['strand'].iloc[row])
        try:
            gene_dict[table['gene_name'].iloc[row]][1].add(table['transcript_id'].iloc[row])

            if table['strand'].iloc[row] == '-':
                gene_dict[table['gene_name'].iloc[row]][0].add(table['end'].iloc[row])

            else:
                gene_dict[table['gene_name'].iloc[row]][0].add(table['start'].iloc[row])
        except KeyError:
            if table['strand'].iloc[row] == '-':
                gene_dict[table['gene_name'].iloc[row]] = ({table['end'].iloc[row]}, {table['transcript_id'].iloc[row]})
            else:
                gene_dict[table['gene_name'].iloc[row]] = ({table['start'].iloc[row]}, {table['transcript_id'].iloc[row]})
             
    return gene_dict


def all_transcripts(gene_dict):
    '''
    '''
    new_dict={}
    for key, value in gene_dict.items():
        #this filter subsets it to genes with only one TSS
        if len(value[1])  > 1:
            new_dict[key] = value
    
    return new_dict
        

def make_df_starts(new_dict):
    '''
    takes a dictionary and generates a df of starts with num_start 
    '''
    #so compare to the set dict of ensembl transcripts 
    start_end_to_index_dict = {} # want to have the start/end coord and also the index number 
    
    ens_ids = []
    TSS_number_list  = []
    gene = []
    ix = 0
    for key, values in new_dict.items():
        TSS_number = 1
        for value in values[0]:
            new_df = ercc_new[ercc['gene_name']  == key]
            for row in range(len(new_df)):
                if new_df['start'].iloc[row]  == value:
                    ix +=1
                    ens_ids.append(new_df['transcript_id'].iloc[row])
                    TSS_number_list.append(TSS_number)
                    gene.append(key)
                elif new_df['end'].iloc[row]  == value: 
                    ix +=1
                    ens_ids.append(new_df['transcript_id'].iloc[row])
                    TSS_number_list.append(TSS_number)
                    gene.append(key)
            TSS_number += 1
                    
    zipped  = list(zip(ens_ids, TSS_number_list, gene))
    df_zip = pd.DataFrame(zipped, columns = ['transcript_id', 'TSS', "gene_name"], dtype = str) 
    return df_zip


def subset_ercc(ercc_new, filtering_list, gene_name = True):
    '''
    take in ERCC filters based on whether its in a list 
    '''
    if gene_name:
        test1 = ercc_new[ercc_new.gene_name.isin(filtering_list)]
    else: 
        test1 = ercc_new[ercc_new.transcript_id.isin(filtering_list)]
    
    return test1
    
def get_coords_bed(bed):
        
        bed = bed[bed[0] ==  "chrX"] 
        v2_start = list(bed[1])
        v3_end = list(bed[2])
        zipped_bed = list(zip(v2_start,v3_end))
        
        return zipped_bed
    
def how_far_are_starts(diction, length_dist):
        '''
        #should take in a dictionary where each gene has more than one diff. start
        make a dict wiht the genes and the starts as a tuple if the TSS is within 100 of each other 
        '''
        #diction = {'EIF1AY': ({20575794, 20575725}, {'ENST00000382772', 'ENST00000361365'})}
        #length_dist = 200
        
        new_dict = {}
        
        for key, value in diction.items():
            #print(key,value, "here")
            if len(value[0]) == 1:
                new_dict[key] = value[0]
                continue
            
            comps = get_right_length_comps(get_all_comps(list(diction[key][0])))
            
           
            if len(comps) == 1:
               # print("HERE BITCH")
                if abs(comps[0][0] - comps[0][1]) < length_dist:
                #    print("ur dumb")
                    new_dict[key] =  {comps[0][1]}
                continue
                   

            to_pop = []
            for test in comps:
                dist = abs(test[0] - test[1])
                if dist < length_dist:
                    to_pop.append(test)
                
            if to_pop:
                
                comp_1 = copy.deepcopy(comps)
                for entry in to_pop:
                    #print(entry)
                    for option in comps:
                    #    print(option)
                        if entry == option: 
                            try:
                                comp_1.remove(entry)
                            except:
                                next
                        else: 
                            entry0 = entry[0]
                            for num in option: 
                               # print(entry0,num, option)
                                if num == entry0:
                                #    print('here')
                                    try:
                                        comp_1.remove(option)
                                    except:
                                        next
                starts = set()
                for val in comp_1: 
                    starts.add(val[0])
                    starts.add(val[1])
            
                new_dict[key] = starts
                
            else:
                 new_dict[key] = value[0]
                                
        return(new_dict)
        

In [90]:
if __name__ == "__main__":
    ercc = pd.read_csv("annotation_with_ERCC.txt",sep=' ')
    #changed some gene names in here#
    genes = ["CMV", "EEF1A1", "ACTB", "TXLNG", "TXLNGY", "EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X", "TMSB4X", "TMSB4Y", "PRKX", "PRKY", "NLGN4X", "NLGN4Y"]

    subsetted_xyp = subset_ercc(ercc, genes, gene_name = True)
    print(subsetted_xyp)
    subsetted_xyp.to_csv("subsetted_ercc_annotation_1.txt",  index=None, sep=str('\t'))
    subsetted_xyp.iloc[:,0:8].to_csv("subsetted_ercc_annotation_1.txt",  index=None, sep=str('\t'))
        
    subsetted_xyp.iloc[:,[0,3,4]].to_csv("subsetted_ercc_annotation.txt",  index=None, sep=str('\t'))

    all_starts_dict = get_all_starts(subsetted_xyp)

    out = how_far_are_starts(all_starts_dict, 200) #something is off with this function - I will just go with the all_starts dict and figure the calculations out from there 
    
   
    
    all_starts_dict['USP9Y'] = ({12662368}, {'mine'}) #new USP9Y start 12,662,368

    all_starts_dict['TMSB4Y'] = ({13703899}, {'mine'})  
    
    all_starts_dict['USP9X'] = ({41085445}, {'mine'}) 
    
    all_starts_dict['CMV'] = ({1}, {'mine'}) 
    
    #using the fantom promoters, going to add those in here as 'mine'
    #from /lab/solexa_page/220516_mpra/helen_start_sites/fantom_promtoers
    
    all_starts_dict['DDX3X'] = ({41334161}, {'fantom'}) 
    all_starts_dict['DDX3Y'] = ({12904881}, {'fantom'}) 
    all_starts_dict['EIF1AX'] = ({20141834}, {'fantom'}) 
    all_starts_dict['EIF1AY'] = ({20575775}, {'fantom'}) 
    all_starts_dict['KDM5C'] = ({53225206}, {'fantom'}) 
    all_starts_dict['KDM5D'] = ({19744725}, {'fantom'}) 
    all_starts_dict['KDM6A'] = ({44873187}, {'fantom'}) 
    all_starts_dict['RPS4X'] = ({72277247}, {'fantom'}) 
    all_starts_dict['RPS4Y1'] = ({2841601}, {'fantom'}) 
    all_starts_dict['USP9X'] = ({41085444}, {'fantom'}) 
    all_starts_dict['USP9Y'] = ({12662352}, {'fantom'}) 
    all_starts_dict['UTY'] = ({13479986}, {'fantom'}) 
    all_starts_dict['ZFX'] = ({24149728}, {'fantom'}) 
    all_starts_dict['ZFY'] = ({2935380}, {'fantom'}) 

print(all_starts_dict['DDX3Y']) 
print(all_starts_dict)

       seqname   source     feature     start       end score strand frame  \
33436     chr6   HAVANA  transcript  73515750  73521018     .      -     .   
33437     chr6  ENSEMBL  transcript  73515752  73521032     .      -     .   
33438     chr6  ENSEMBL  transcript  73515752  73521032     .      -     .   
33439     chr6   HAVANA  transcript  73517515  73521588     .      -     .   
33440     chr6   HAVANA  transcript  73517521  73521721     .      -     .   
...        ...      ...         ...       ...       ...   ...    ...   ...   
100970    chrY   HAVANA  transcript  19705415  19744923     .      -     .   
100971    chrY  ENSEMBL  transcript  19705417  19744939     .      -     .   
100972    chrY   HAVANA  transcript  19705420  19744761     .      -     .   
100979    chrY   HAVANA  transcript  20575725  20593154     .      +     .   
100980    chrY   HAVANA  transcript  20575794  20592630     .      +     .   

                gene_id gene_name       gene_type    transcript

In [91]:
    list_gene = [] 
    transcript = [] 
    list_starts = [] 
    
    for key, val in all_starts_dict.items(): 
        for start in val[0]:
            list_gene.append(key)
            list_starts.append(start)
            to_add = key + "_" + str(start)
            transcript.append(to_add)
            
    zipped = list(zip(list_gene, list_starts, transcript))
    df_zip = pd.DataFrame(zipped,  columns = ["gene.x","starts_tx", "transcript"], dtype = str) 
    print(df_zip)
    df_zip.to_csv("starts.csv")
    
    #print(df)

    gene.x starts_tx       transcript
0   EEF1A1  73521032  EEF1A1_73521032
1   EEF1A1  73521721  EEF1A1_73521721
2   EEF1A1  73521018  EEF1A1_73521018
3   EEF1A1  73521588  EEF1A1_73521588
4     ACTB   5530709     ACTB_5530709
5     PRKX   3713608     PRKX_3713608
6   NLGN4X   6151458   NLGN4X_6151458
7   NLGN4X   6227203   NLGN4X_6227203
8   NLGN4X   6227847   NLGN4X_6227847
9   NLGN4X   6228615   NLGN4X_6228615
10  NLGN4X   6228863   NLGN4X_6228863
11  TMSB4X  12975244  TMSB4X_12975244
12  TMSB4X  12975658  TMSB4X_12975658
13  TMSB4X  12975108  TMSB4X_12975108
14  TMSB4X  12975110  TMSB4X_12975110
15   TXLNG  16786432   TXLNG_16786432
16   TXLNG  16786427   TXLNG_16786427
17  EIF1AX  20141834  EIF1AX_20141834
18     ZFX  24149728     ZFX_24149728
19   USP9X  41085444   USP9X_41085444
20   DDX3X  41334161   DDX3X_41334161
21   KDM6A  44873187   KDM6A_44873187
22   KDM5C  53225206   KDM5C_53225206
23   RPS4X  72277247   RPS4X_72277247
24  RPS4Y1   2841601   RPS4Y1_2841601
25     ZFY  

In [93]:
   
filename_list = ['lcl_50_1.txt', 'fib_1_50_1.txt', 'fib_2_50_1.txt', 'full_fib_50_1.txt', 'fib_1_200_1.txt', 
                'fib_2_200_1.txt', 'full_fib_200_1.txt']
filename_list = ['lcl_200_1.txt']

    
for filename in filename_list:    
    
    df = pd.read_csv(filename, sep = "\t") #     df = pd.read_csv("fib_counts_1_200.txt", sep = "\t") # 

    df[['chr', 'nums']] = df['coords'].str.split(":", expand = True) 
    
    df[['start', 'end']] = df['nums'].str.split("-", expand = True) 
    
    
    
    merged = pd.merge(df_zip, df, on='gene.x')
    #print(df)
    merged_minus = merged[merged['orientation.x'] == 'minus']
    
    merged_minus['start'] = merged_minus["start"].astype("float64")  
    merged_minus['starts_tx'] = merged_minus["starts_tx"].astype("float64") 
    merged_minus['distance'] = (merged_minus['start'] - merged_minus['starts_tx'])*(-1) #bc on the - strand

    merged_plus = merged[merged['orientation.x'] == 'plus']
    
    merged_plus['start'] = merged_plus["start"].astype("float64") 
    merged_plus['starts_tx'] = merged_plus["starts_tx"].astype("float64") 
    merged_plus['distance'] = merged_plus['start'] - merged_plus['starts_tx'] 

    #type(merged_minus)
    concat_merged_dis = pd.concat([merged_minus, merged_plus])
    #print(concat_merged_dis)
    concat_merged_dis.to_csv("concat_merge_dist_" + filename + ".csv")
    

/tmp/ipykernel_3779053/1851959017.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_minus['start'] = merged_minus["start"].astype("float64")
/tmp/ipykernel_3779053/1851959017.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_minus['starts_tx'] = merged_minus["starts_tx"].astype("float64")
/tmp/ipykernel_3779053/1851959017.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

In [83]:
print(concat_merged_dis)

      gene.x   starts_tx       transcript                        coords  \
0     EEF1A1  73521032.0  EEF1A1_73521032        chr6:73520747-73520946   
1     EEF1A1  73521032.0  EEF1A1_73521032  chr6:73520747-73520946_shuff   
2     EEF1A1  73521032.0  EEF1A1_73521032        chr6:73520797-73520996   
3     EEF1A1  73521032.0  EEF1A1_73521032  chr6:73520797-73520996_shuff   
4     EEF1A1  73521032.0  EEF1A1_73521032        chr6:73520847-73521046   
...      ...         ...              ...                           ...   
3182     CMV         1.0            CMV_1             cmv:251-450_shuff   
3183     CMV         1.0            CMV_1                   cmv:301-500   
3184     CMV         1.0            CMV_1             cmv:301-500_shuff   
3185     CMV         1.0            CMV_1                    cmv:51-250   
3186     CMV         1.0            CMV_1              cmv:51-250_shuff   

          log2 orientation.x          type   chr                     nums  \
0    -0.860806        

In [50]:
#load the chosen start sites 
#does the ERCC have more information about the exons/introns etc. 

ercc_small = ercc[ercc.gene_name.isin(genes)]

#transcript_starts_list = ["DDX3X_41334162","DDX3Y_12904831", "EIF1AX_20141844","EIF1AY_20575725", "KDM5C_53225422" ,"KDM6A_44873177" , "NLGN4X_6228863" ,
#                          "NLGN4Y_14524574" , "RPS4X_72277300" ,"RPS4Y1_2841486",  "TXLNG_16786432" ,"TXLNGY_19567382" ,"UTY_13479978",
#                           "ZFX_24149790","ZFY_2935389", "KDM5D_19744923"] #,"USP9Y_12662368", "USP9X_41085445" "TMSB4X_12975108", "TMSB4Y_13703899" *leaving out because changed the starts so I need to adjust this later
#new_frame = pd.DataFrame()
#new_frame['gene_start'] = transcript_starts_list
#new_frame['chosen_start'] = 'yes'
#ercc['gene_start'] = ercc['gene_name'].astype(str) + "_" + ercc["start"].astype(str)


In [45]:
tx_names = ercc[ercc.gene_start.isin(transcript_starts_list)]

tx_names = tx_names['transcript_name']

In [22]:
import pyranges as pr
import subprocess 


#gt gff3 -retainids -addintrons Homo_sapiens.GRCh38.110.chromosome.X.gff3 > Homo_sapiens.GRCh38.110.chromosome.X.wintrons.gff3


CompletedProcess(args=['gt', 'gff3', '-retainids', '-addintrons', '/nfs/genomes/human_hg38_dec13_no_random/gtf/Homo_sapiens.GRCh38.93.chr.canonical.gff3', '-o', 'Homo_sapiens.GRCh38.93.chr.canonical.plusintrons.gff3'], returncode=1, stderr=b'gt gff3: error: line 1 in file "/nfs/genomes/human_hg38_dec13_no_random/gtf/Homo_sapiens.GRCh38.93.chr.canonical.gff3" does not begin with "##gff-version" or "##gff-version"\n')

## getting gtf for genes for alignment uses

In [1]:
gtf_ = pr.read_gtf("/nfs/genomes/human_hg38_dec13_no_random/gtf/Homo_sapiens.GRCh38.99.chr.canonical.gtf")
gtf_ = gtf_.df

NameError: name 'pr' is not defined

In [25]:
#gtf_.columns = ['seqname','source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute' ]
genes = ["TXLNG", "TXLNGY", "EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X", "TMSB4X", "TMSB4Y", "NLGN4X", "NLGN4Y"] 

gtf_xy = gtf_[gtf_.gene_name.isin(genes)]

#extract the exons and genes
#gtf_xy = gtf_xy[(gtf_xy['Feature'] == "gene") | (gtf_xy['Feature'] == "exon")]
gtf_xy = gtf_xy[(gtf_xy['Feature'] == "CDS") | (gtf_xy['Feature'] == "exon")]

gtf_xy['Feature'].unique() 
gtf_xy['gene_name'].unique()
gtf_xy['Feature'].unique() 

array(['exon', 'CDS'], dtype=object)

In [39]:
#there could be an issue with the start site of one affecting the rest of the calculations, cannot have that be the case
#need to have a cutoff for the transcript support level 
#gtf_xy = gtf_xy[(gtf_xy['transcript_support_level'].astype(int) > 2)] # three or greater 
#gtf_xy['transcript_support_level'] = gtf_xy['transcript_support_level'].replace('', 0).replace('NA', 0).astype(int)
#gtf_xy = gtf_xy[(gtf_xy['transcript_support_level'].astype(int) >= 1)] #keeping all of the possible transcripts
#gtf_.loc[gtf_['gene_id'] == "ENSG00000154620"]["gene_name"]


#Homo_sapiens.GRCh38.93.chr.canonical.gff3

gtf_xy[gtf_xy['gene_name'] == "TXLNGY"].iloc[1:20]

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_version,...,tag,transcript_support_level,exon_number,exon_id,exon_version,(assigned,previous,protein_id,protein_version,ccds_id
2898629,chrY,havana,exon,19587209,19587507,.,+,.,ENSG00000131002,12,...,basic,3,2,ENSE00002783311,1,NaN,NaN,NaN,NaN,NaN
2898630,chrY,havana,exon,19588369,19588650,.,+,.,ENSG00000131002,12,...,basic,3,3,ENSE00002873804,1,NaN,NaN,NaN,NaN,NaN
2898631,chrY,havana,exon,19589526,19589612,.,+,.,ENSG00000131002,12,...,basic,3,4,ENSE00002825272,1,NaN,NaN,NaN,NaN,NaN
2898632,chrY,havana,exon,19590082,19590247,.,+,.,ENSG00000131002,12,...,basic,3,5,ENSE00001893165,2,NaN,NaN,NaN,NaN,NaN
2898634,chrY,havana,exon,19567349,19567570,.,+,.,ENSG00000131002,12,...,NaN,2,1,ENSE00001826433,1,NaN,NaN,NaN,NaN,NaN
2898635,chrY,havana,exon,19587209,19587507,.,+,.,ENSG00000131002,12,...,NaN,2,2,ENSE00002783311,1,NaN,NaN,NaN,NaN,NaN
2898636,chrY,havana,exon,19588369,19589612,.,+,.,ENSG00000131002,12,...,NaN,2,3,ENSE00001875276,1,NaN,NaN,NaN,NaN,NaN
2898637,chrY,havana,exon,19590082,19590418,.,+,.,ENSG00000131002,12,...,NaN,2,4,ENSE00002811240,1,NaN,NaN,NaN,NaN,NaN
2898639,chrY,havana,exon,19567381,19567482,.,+,.,ENSG00000131002,12,...,NaN,2,1,ENSE00002957208,1,NaN,NaN,NaN,NaN,NaN
2898640,chrY,havana,exon,19587209,19587507,.,+,.,ENSG00000131002,12,...,NaN,2,2,ENSE00002783311,1,NaN,NaN,NaN,NaN,NaN


In [72]:
def intron_overlapping(exons, intron_start, intron_end):
    #print(intron_start, intron_end, "intron_coords")
    
    
    for index in range(len(exons)): 
        #print(exons.iloc[index]['Start'], 'start')
        #print(exons.iloc[index]['End'], 'end')
#    print(exons.iloc[index])
        exon_st = exons.iloc[index]['Start']
        exon_end = exons.iloc[index]['End']
       # print(((exon_st <= intron_end) & (exon_end >= intron_end)), ((exon_st <= intron_start) & (exon_end >= intron_start)), exon_st, exon_end, intron_start, intron_end)
        
        if ((exon_st >= intron_start) & (exon_st <= intron_end)):
            return True
        if ((exon_end >= intron_start) & (exon_end <= intron_end)):
            return True
        
        
        if ((exon_st <= intron_start) & (exon_end >= intron_start)):
            return True 
        if ((exon_st <= intron_end) & (exon_end >= intron_end)):
            return True
        #if ((exon_st >= intron_start) & (exon_end <= intron_start)):
        #    return True
        #if ((exon_st >= intron_end) & (exon_end <= intron_end)):
        #    return True
    return False


In [73]:
intron_coordinates = pd.DataFrame(columns=['seqname', 'feature', 'start', 'end', 'gene_name'])  
genes = ["TXLNG", "TXLNGY", "EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X", "TMSB4X", "TMSB4Y", "NLGN4X", "NLGN4Y"] 

#genes = ["ZFY"] 

for gene in genes: 
    testzfx = gtf_xy[gtf_xy['gene_name'] == gene] 
    exons = testzfx[testzfx['Feature'] == "CDS"]
    print(exons)
    if gene == 'TXLNGY':
        exons = testzfx[testzfx['Feature'] == "exon"]
    exons = exons.sort_values(['Chromosome', 'Start', 'End']) #sort values start to end 
    grouped_exons = exons.groupby('transcript_id')
    print(grouped_exons)
    for name, group in grouped_exons:
      #  print(name)
        for i in range(1, len(group)):
            intron_start = group.iloc[i-1][4] 
            intron_end = group.iloc[i][3] 
            
            if (intron_end - intron_start > 200) & (intron_end > intron_start): 
                intron_start = intron_start + 100
                intron_end = intron_end - 100
            else:
                continue
        # Check for overlap with exons of any transcripts 
            #print(group.iloc[i-1][24] == '1')  
            #get rid of any grouped exon 
            if (group.iloc[i-1][24] == '1') | (group.iloc[i][24] == '1'): 
                print('FIRST', intron_start, intron_end)
                continue
            
            if intron_overlapping(exons, intron_start, intron_end): #this doesnt crop out everything that id want it to# #need to make it better                 
                print("HERE", intron_start, intron_end) 
                continue
            
            else:
                print("adding", intron_start, intron_end) 
                new_row = pd.Series({'seqname': testzfx['Chromosome'].iloc[0], 'feature': 'intron', 'start': intron_start, 'end':intron_end, 'gene_name':gene}) 
                intron_coordinates = intron_coordinates.append(new_row, ignore_index=True) 
                

        Chromosome          Source Feature     Start       End Score Strand  \
2808006       chrX         ensembl     CDS  16786487  16786589     .      +   
2808009       chrX         ensembl     CDS  16828093  16828264     .      +   
2808011       chrX         ensembl     CDS  16829575  16829770     .      +   
2808013       chrX         ensembl     CDS  16832622  16832742     .      +   
2808015       chrX         ensembl     CDS  16834282  16834357     .      +   
2808017       chrX         ensembl     CDS  16837592  16837685     .      +   
2808019       chrX         ensembl     CDS  16839820  16839916     .      +   
2808021       chrX         ensembl     CDS  16841427  16841763     .      +   
2808027       chrX  ensembl_havana     CDS  16786487  16786589     .      +   
2808030       chrX  ensembl_havana     CDS  16818573  16818877     .      +   
2808032       chrX  ensembl_havana     CDS  16820163  16820255     .      +   
2808034       chrX  ensembl_havana     CDS  16828093

/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

adding 16837785 16839720
adding 16840016 16841327
Empty DataFrame
Columns: [Chromosome, Source, Feature, Start, End, Score, Strand, Frame, gene_id, gene_version, gene_name, gene_source, gene_biotype, transcript_id, transcript_version, transcript_name, transcript_source, transcript_biotype, tag, transcript_support_level, exon_number, exon_id, exon_version, (assigned, previous, protein_id, protein_version, ccds_id]
Index: []

[0 rows x 28 columns]
HERE 19592914 19593298
HERE 19597765 19599638
adding 19599931 19602115
adding 19602411 19603696
HERE 19567582 19587109
HERE 19587607 19588269
HERE 19588750 19589426
HERE 19589712 19589982
HERE 19567582 19587109
HERE 19587607 19589420
HERE 19589712 19591679
HERE 19592059 19593298
HERE 19593693 19595914
HERE 19596234 19597490
HERE 19597765 19599638
adding 19599931 19602115


/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

adding 19602411 19603696
HERE 19568051 19587109
HERE 19567582 19587109
HERE 19587607 19589420
HERE 19589712 19589982
HERE 19567670 19587109
HERE 19587607 19588269
HERE 19589712 19589982
adding 19590872 19591679
HERE 19592059 19593298
adding 19568051 19569284
adding 19569559 19587109
HERE 19587607 19588269
HERE 19588750 19589420
HERE 19589712 19591819
HERE 19592059 19593298
HERE 19588750 19589420
HERE 19589712 19593298
HERE 19588750 19589420
HERE 19589712 19591679
HERE 19592059 19593298
HERE 19588750 19589420
HERE 19589712 19589982
adding 19597765 19598451
adding 19598739 19599638


...

/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

adding 6029532 6032590
adding 6032850 6150894
        Chromosome          Source Feature     Start       End Score Strand  \
2898190       chrY          havana     CDS  14622119  14622591     .      +   
2898193       chrY          havana     CDS  14719458  14719518     .      +   
2898195       chrY          havana     CDS  14723116  14723269     .      +   
2898197       chrY          havana     CDS  14824187  14824373     .      +   
2898199       chrY          havana     CDS  14829729  14830519     .      +   
2898201       chrY          havana     CDS  14840412  14841259     .      +   
2898213       chrY  ensembl_havana     CDS  14723148  14723269     .      +   
2898216       chrY  ensembl_havana     CDS  14824187  14824373     .      +   
2898218       chrY  ensembl_havana     CDS  14829729  14830519     .      +   
2898220       chrY  ensembl_havana     CDS  14840412  14841259     .      +   
2898229       chrY  ensembl_havana     CDS  14622119  14622591     .      +   
289823

/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

adding 14830619 14840312
HERE 14723369 14824087
adding 14824473 14829629
adding 14830619 14840312
adding 14622691 14719358
adding 14719618 14723016
adding 14723369 14733351
adding 14622691 14719358
adding 14719618 14723016
HERE 14723369 14824087
adding 14824473 14829629
adding 14830619 14840312


/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=True)
/tmp/ipykernel_3779053/2834357883.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  intron_coordinates = intron_coordinates.append(new_row, ignore_index=Tr

In [74]:
gtf_small = gtf_xy[['Chromosome', 'Feature', 'Start', 'End', 'gene_name']] 
gtf_small = gtf_small.rename(columns={"Chromosome": "seqname", "Feature": "feature", "Start":"start", "End": "end", "gene_name": "gene_name"}) #("seqname", "feature", "start", "end", "gene_name")
gtf_small[gtf_small['gene_name'] == 'TMSB4Y']

,seqname,feature,start,end,gene_name
2898127,chrY,exon,13703898,13704435,TMSB4Y
2898128,chrY,CDS,13704335,13704435,TMSB4Y
2898130,chrY,exon,13705224,13706024,TMSB4Y
2898131,chrY,CDS,13705224,13705256,TMSB4Y


In [75]:
vertically_concatenated = pd.concat([gtf_small, intron_coordinates], axis=0) 
vertically_concatenated = vertically_concatenated[(vertically_concatenated['feature'] == "CDS") | (vertically_concatenated['feature'] == "intron") | ((vertically_concatenated['feature'] == 'exon') & (vertically_concatenated['gene_name'] == 'TXLNGY'))]
print(set(vertically_concatenated['gene_name']))

{'UTY', 'RPS4X', 'DDX3X', 'EIF1AX', 'ZFY', 'TMSB4Y', 'TMSB4X', 'ZFX', 'KDM6A', 'TXLNG', 'USP9X', 'TXLNGY', 'NLGN4X', 'KDM5D', 'DDX3Y', 'USP9Y', 'EIF1AY', 'KDM5C', 'RPS4Y1', 'NLGN4Y'}


In [76]:
vertically_concatenated['feature'].unique()
#get only unique rows ####
vertically_concatenated.to_csv("XY_gtf_w_introns_100923.txt",  index=None, sep=str('\t'))
#vertically_concatenated.to_csv("XY_gtf_w_introns_072123.txt",  index=None, sep=str('\t'))

In [77]:
#gtf_small.to_csv("/lab/solexa_page/hannah/2XY_gtf_small_072123.txt",  index=None, sep=str('\t'))
vertically_concatenated[vertically_concatenated['gene_name'] == 'TXLNGY']

,seqname,feature,start,end,gene_name
2898628,chrY,exon,19567312,19567482,TXLNGY
2898629,chrY,exon,19587209,19587507,TXLNGY
2898630,chrY,exon,19588369,19588650,TXLNGY
2898631,chrY,exon,19589526,19589612,TXLNGY
2898632,chrY,exon,19590082,19590247,TXLNGY
...,...,...,...,...,...
21,chrY,intron,19569559,19587109,TXLNGY
22,chrY,intron,19597765,19598451,TXLNGY
23,chrY,intron,19598739,19599638,TXLNGY
24,chrY,intron,19599931,19602115,TXLNGY


In [273]:
from Bio.Align.Applications import ClustalOmegaCommandline
in_file = "lcl_starts_bed_1000bp_1019.fa"
out_file = "aligned.fasta"
clustalomega_cline = ClustalOmegaCommandline(infile=in_file, outfile=out_file, verbose=True, auto=True) 
print(clustalomega_cline)
#clustalo -i unaligned.fasta -o aligned.fasta --auto -v

clustalo -i lcl_starts_bed_1000bp_1019.fa -o aligned.fasta --auto -v


In [ ]:
clustalomega_cline()

In [2]:
#R code to extract sequences NEED TO CHANGE CODE TO R
library('BSgenome') 



Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The

In [5]:

genome_hs <- getBSgenome("BSgenome.Hsapiens.UCSC.hg38")
genome_pantro <- getBSgenome("BSgenome.Ptroglodytes.UCSC.panTro3")
genome_mmulatta <- getBSgenome("BSgenome.Mmulatta.UCSC.rheMac10")
genome_caljac <- getBSgenome("BSgenome.Cjacchus.UCSC.calJac3") 


Bioconductor version '3.10' is out-of-date; the current release version '3.16'
  is available with R version '4.2'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cloud.r-project.org


Bioconductor version 3.10 (BiocManager 1.30.16), R 3.6.3 (2020-02-29)

Installing package(s) 'BSgenome.Ptroglodytes.UCSC.panTro3'

Installation paths not writeable, unable to update packages
  path: /nfs/apps/lib/R/3.6/site-library.2022q1
  packages:
    additivityTests, ade4, AER, akima, alphahull, amap, Amelia, aod, ape,
    argparse, arm, aroma.affymetrix, aroma.apd, aroma.cn, aroma.core, arrow,
    ashr, babelgene, base64, BatchJobs, batchtools, bayesm, BBmisc, bbmle,
    bdsmatrix, beanplot, BH, BiasedUrn, bibtex, bigmemory, bigmemory.sri,
    BiocManager, bit, blob, blockmodeling, BMA, boot, brew, broom, bslib,
    Cairo, callr, canvasXpress, car, caret, chron, circl

ERROR: Error in .stopOnAvailablePkg(genome): BSgenome.Mmulatta.UCSC.rheMac10 package is not currently installed.
  You first need to install it, which you can do with:
      library(BiocManager)
      install("BSgenome.Mmulatta.UCSC.rheMac10")


In [25]:
df = pd.read_csv("file_for_fifty_bp_dist_calcs_k562.txt", sep = "\t") 



In [26]:
print(df)

                      coords      log2  gene.x orientation.x
1     chr6:73520747-73520797 -1.034596  EEF1A1         minus
2     chr6:73520797-73520847 -0.998709  EEF1A1         minus
3     chr6:73520847-73520897 -1.053027  EEF1A1         minus
4     chr6:73520897-73520947  0.077554  EEF1A1         minus
5     chr6:73520947-73520997  1.473086  EEF1A1         minus
...                      ...       ...     ...           ...
1797             cmv:251-301  1.310459     CMV          plus
1798             cmv:301-351  1.789657     CMV          plus
1799             cmv:351-401  1.813127     CMV          plus
1800             cmv:401-451  1.686615     CMV          plus
1801             cmv:451-501  2.098418     CMV          plus

[1801 rows x 4 columns]


In [27]:
df = pd.read_csv("fib_counts_1_200.txt", sep = "\t") #


In [28]:
df

,coords,log2,gene.x,orientation.x,type
0,chr6:73520747-73520946,-0.874478,EEF1A1,minus,positive ctl
1,chr6:73520747-73520946_shuff,-1.500939,EEF1A1,minus,negative ctl
2,chr6:73520797-73520996,-1.090219,EEF1A1,minus,positive ctl
3,chr6:73520797-73520996_shuff,-0.842316,EEF1A1,minus,negative ctl
4,chr6:73520847-73521046,-0.930851,EEF1A1,minus,positive ctl
...,...,...,...,...,...
1884,cmv:251-450_shuff,-0.348521,CMV,plus,negative ctl
1885,cmv:301-500,4.948123,CMV,plus,positive ctl
1886,cmv:301-500_shuff,-0.303872,CMV,plus,negative ctl
1887,cmv:51-250,6.783378,CMV,plus,positive ctl


In [29]:
df = pd.read_csv("fib_counts_1_200.txt", sep = "\t") #

In [30]:
df

,coords,log2,gene.x,orientation.x,type
0,chr6:73520747-73520946,-0.874478,EEF1A1,minus,positive ctl
1,chr6:73520747-73520946_shuff,-1.500939,EEF1A1,minus,negative ctl
2,chr6:73520797-73520996,-1.090219,EEF1A1,minus,positive ctl
3,chr6:73520797-73520996_shuff,-0.842316,EEF1A1,minus,negative ctl
4,chr6:73520847-73521046,-0.930851,EEF1A1,minus,positive ctl
...,...,...,...,...,...
1884,cmv:251-450_shuff,-0.348521,CMV,plus,negative ctl
1885,cmv:301-500,4.948123,CMV,plus,positive ctl
1886,cmv:301-500_shuff,-0.303872,CMV,plus,negative ctl
1887,cmv:51-250,6.783378,CMV,plus,positive ctl


In [25]:
#genes = ["TXLNG", "TXLNGY", "EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY",  "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X", "TMSB4X", "TMSB4Y"] 
#genes = ["EIF1AY"] 
genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY",  "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X", "TMSB4X", "TMSB4Y"] 

#genes = ["ZFX"]

types_of_seq = ["upmax", "downmax"] #"exon", "intron", "max_peak"] 
from Bio import AlignIO 
from Bio import SeqIO 

#genes = ["TXLNGY"] 
#genes = ["ZFX"] 
for gene in genes: 
    for type_1 in types_of_seq: 
        #alignments = AlignIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/by_feature/' +  gene + type_1 + '_msa.fa', "fasta")
        #print(alignments)
        #print([record for record in alignments]) # if len(record.seq) > 0])
    # Create a new list of non-empty alignment records
        #non_empty_alignments = [record for record in alignments if len(record.seq) > 0]
        #print(non_empty_alignments)
        name = '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/by_feature/' + gene + type_1 + '_msa.new.fa'
    # Write the non-empty alignments to the output file in PHYLIP format
       # AlignIO.write(non_empty_alignments, name, 'phylip') 
        
        with open(name, 'w') as name:

            for record in SeqIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/by_feature/' +  gene + type_1 + '_msa.fa', "fasta"):
        # Check if the sequence is empty
                print(len(record.seq))
                if len(record.seq) == 0:
            # Skip this record and don't write it to the output file
                    print("here")
                    continue
       # Write the record to the output file
                SeqIO.write(record, name, 'fasta')
       # print("hello") 
        name1 = '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/by_feature/' + gene + type_1 + '_msa.new.fa'
        records = SeqIO.parse(name1, "fasta")
        
        new_name = '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/by_feature/' +  gene + type_1 + '_msa.phy'

        SeqIO.write(records, new_name, 'phylip')
      #print(gene) 
      #alignments = AlignIO.parse('/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/by_feature/' +  gene + type_1 + '_msa.fa', "fasta")
      #i should be able to get rid of groups with nothing there? 
      #  AlignIO.write(alignments, name, 'phylip') 
    

200
200
200
200
231
231
231
231
856
856
856
856
203
203
203
203
205
205
205
205
207
207
207
207
200
200
200
200
202
202
202
202
202
202
202
202
202
202
202
202
200
200
200
200
200
200
200
200
204
204
204
204
200
200
200
200
210
210
210
210
201
201
201
201
200
200
200
200
230
230
230
230
211
211
211
0
here
200
200
200
0
here
202
202
202
0
here
205
205
205
0
here
367
367
367
367
229
229
229
229
200
200
200
200
206
206
206
206
200
200
200
200
217
217
217
217
200
200
200
200
200
200
200
200


In [26]:
#gene = "NLGN" 
#name1 = '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/nlgn4_analysis/' + gene +  '_msa_new.fa' 

name1 = '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/nlgn4_analysis/out.fa'
records = SeqIO.parse(name1, "fasta") 
new_name = '/lab/solexa_page/hannah/220516_mpra/msa/long_alignments/nlgn4_analysis/' +  'out' + '_msa.phy' 

SeqIO.write(records, new_name, 'phylip') 

3

In [12]:
AlignIO.write(alignments, name, 'phylip') 

0

In [63]:
result = subprocess.run([ "mkdir",  "test1"], check = True)  

print(result.stdout) 

None


In [64]:
subprocess.run(["rm", "-rf", "test1"], check=True)

CompletedProcess(args=['rm', '-rf', 'test1'], returncode=0)

In [13]:
import gtfparse

In [12]:
pip install gtfparse

/usr/local/lib/python3.8/dist-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/local/lib/python3.8/dist-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/jupyterhub/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
from gtfparse import read_gtf



In [11]:
read_gtf

<function gtfparse.read_gtf.read_gtf(filepath_or_buffer, expand_attribute_column=True, infer_biotype_column=False, column_converters={}, usecols=None, features=None, result_type='polars')>